In [19]:
from requests import get
from json import loads
from services.db_service import DbService

In [20]:
db_service = DbService()

In [21]:
# name, color, colors, color_identity, cmc, released_at, set, banned
def get_card_data_from_scryfall(name):
    base_url = "https://api.scryfall.com"
    scryfall_url = f"{base_url}/cards/search?q=!\"{name}\" is:firstprint"
    
    card = loads(get(scryfall_url).text)["data"][0]

    scryfall_name = card["name"]
    cmc = card["cmc"]
    color_identity = card["color_identity"]
    released_at = card["released_at"]
    set = card["set_name"]

    legality = card["legalities"]["pauper"]
    if legality == "not_legal":
        banned = True
    else:
        banned = False
        
    if "colors" in card:
        colors = card["colors"]
    else:
        colors = card["card_faces"][0]["colors"]

    type = card["type_line"]
    if "Land" in type:
        color = "L"
    elif len(colors) == 0:
        color = "C"
    elif len(colors) > 1:
        color = "M"
    else:
        color = colors[0]

    return scryfall_name, color, colors, color_identity, cmc, released_at, set, banned

In [22]:
pool_collection = db_service.pool_collection()


def add_card_or_update(card):
    name = card["name"]

    existing_card = pool_collection.find_one({"name": name})
    if existing_card and existing_card["count"] == card["count"]:
        return

    scryfall_name, color, colors, color_identity, cmc, released_at, set, banned = get_card_data_from_scryfall(name)

    if existing_card and existing_card["count"] != card["count"]:
        pool_collection.update_one({"_id": existing_card["_id"]}, {"$set": {"name": name, "count": card["count"], "color": color, "scryfall_name": scryfall_name, "cmc": cmc, "colors": colors, "color_identity": color_identity, "released_at": released_at, "set": set, "banned": banned}})
        return

    pool_collection.insert_one(
        {"name": name, "count": card["count"], "color": color, "scryfall_name": scryfall_name, "cmc": cmc, "colors": colors, "color_identity": color_identity, "released_at": released_at, "set": set, "banned": banned})

In [23]:
aggregate_query = [
    {"$match": {"banned": {"$ne": True}}},
    {"$project": {"all_cards": {"$concatArrays": ["$maindeck", "$sideboard"]}}},
    {"$unwind": "$all_cards"},
    {"$group": {"_id": {"deckId": "$_id", "name": "$all_cards"}, "count": {"$sum": 1}}},
    {"$group": {"_id": "$_id.name", "count": {"$max": "$count"}}},
    {"$sort": {"_id": 1}},
    {"$project": {"name": "$_id", "count": "$count", "_id": 0}}
]

In [24]:
deck_collection = db_service.deck_collection()

cards = list(deck_collection.aggregate(aggregate_query))
for card in cards:
    if card["name"] == "Unknown Card":
        print("OI")
        continue

    add_card_or_update(card)